# hashmap

hashmaps são coleções de chaves e valores.

não importa quantos itens tiver, sempre que precisar acessar uma chave, a complexidade será sempre O(1).  
sabendo que a maior parte dos hashmaps usam estruturas de dados que assemelham-se com arrays, ou são arrays mesmo. uma pergunta pode surgir no meio do caminho:

**como que as entradas são possíveis ser encontradas dentro do array em tempo constante?**

a mágica são as **hash functions**, um dos componentes principais de um hashmap.

a maior parte das implementações de hashmaps usam um (ou mais) array(s) para salvar em memória os dados do hashmap.  
sabendo disso, a função da **hash function** é de mapear as chaves para o memória em si que cada chave aponta.

<img src="public/hash_table.png" width="100%" />

o funcionamento é mais ou menos parecido com isso daqui:

```rust
// create a fresh hashmap; and access a random key
fn main() {
    let hm = HashMap::new();
    hm.get("keytobehashed");
}

// inside the hashmap implementation, we'll find the following behavior
fn get(&self, key: String) {
    let index = self.hash(key);
    return self.array[index];
}
```

sabendo disso a implementação da **hash functions** é muito importante.  
por que ela será responsável por controlar colisões dentro do hashmap, assim evitando dados de serem sobrescritos.

por exemplo uma implementação muito ingênua de uma **hash function** poderia ser algo parecido com isso.

```rust
fn hash(&self, key: String) {
    return key.len();
}
```

nesse caso, o índice retornado para as chaves: `"sp"` e `"rj"` seria igual ₋ já que ambos tem o comprimento igual a `2` que mapearia para o índice `self.array[1]`, logo um sobrescreveria o outro, o que fundamentalmente é uma colisão.

## load factor

_**load factor**_ ou **fator de carga** é a proporção entre espaço total e espaço utilizado em um array.

o **load factor** funciona como um limiar que vai desencadear o redimensionamento do armazenamento do hashmap.  
esse redimensionamento também chamado de **rehashing** é quando o limiar é atingido e o array em que todos os itens estão cresce em tamanho com base em um **growth factor**, que podemos assumir como `2` para simplificar — o que quer dizer que toda vez que se atingir este limiar, o array por baixo vai dobrar de tamanho, e todos os itens serão movidos para essa nova localidade em memória. 

esse mecanismo em conjunto com as **hash functions** trabalham em conjunto para minimizar a possiblidade de colisões dentro de um hashmap.

a ideia de implementar o _**mecanismo de resizing baseado no load factor**_ é que matematicamente, quanto mais o espaço disponível do hashmap é utilizado, naturalmente é maior a probabilidade de colisões.

o valor do _**load factor**_ geralmente gira em torno de `±0.7`, existem muitas pesquisas que apontam que o **load factor** ideal para a maioria dos casos gira em torno desse valor.

## colisões e open addressing

**colisões** dentro de um hashmap acontecem, quando a **hash function** produz o mesmo output para dois valores diferentes, fazendo que os mesmos sejam inseridos no mesmo espaço da memória.

por um certo tempo na história, para lidar com colisões a abordagem era de simplesmente jogar o item para o próximo slot disponível.  
só que isso traz um grande problema.

por que no melhor dos casos a complexidade temporal seria `O(1)` que é caso não haja nenhuma colisão e se pode inserir ou ler o determinado elemento.  
e no pior dos casos a complexidade temporal seria `O(n)`, caso haja uma colisão e o próximo slot não esteja disponível, seria disponível ir até o final do array para inserir o item.

e essa abordagem se chama **open addressing** e nesse caso com **linear probing** que é ir progressivamente aumentando em 1 até encontrar um slot:  
`(hash + 1) % size, (hash + 2) % size, (hash + 3) % size, (hash + 4) % size, ...`

mas existem outras formas de resolver isso ainda usando **open addressing**, o Rust é um ótimo exemplo disso, e usa **quadratic probing**.

> ```rust
> pub struct HashMap<K, V, S = RandomState> { /* private fields */ }
> ```
> 
> A hash map implemented with **quadratic probing** and SIMD lookup.
>
> Link: https://doc.rust-lang.org/std/collections/struct.HashMap.html

a ideia é que ao invés de seguir linearmente, se procura nos slots de intervalos quadráticos.

`(hash + 1^2) % size, (hash + 2^2) % size, (hash + 3^2) % size, (hash + 4^2) % size, (hash + 5^2) % size, ...`

## colisões e chaining

uma alternativa diferente e comumente usada para quando se acontece uma colisão, é trocar a estrutura de dados do slot, de um item único para uma subcoleção, como um **array**, **linked list** ou em outros casos até **trees** como em Java.

<img src="public/hash_map_chaining.png" width="100%" />

é importante notar que usando essa abordagem, quando acessa um item em que houve uma colisão, o acesso deixa de ser `O(1)` e se torna `O(n)`, aonde `N` é o tamanho da subcoleção.

# hashmaps em python

python tem várias implementações de hashmaps na stdlib:

1. `dict` — o dicionário padrão
2. `collections.defaultdict` — dict com valor padrão para chaves inexistentes
3. `collections.Counter` — dict especializado para contagem
4. `set` — conjunto (só chaves, sem valores)

## dict — o básico

In [1]:
# criando um dict
pessoa = {"nome": "João", "idade": 25, "cidade": "São Paulo"}

# acesso — O(1)
print(f"nome: {pessoa['nome']}")

# acesso seguro com get (retorna None se não existir)
print(f"email: {pessoa.get('email')}")
print(f"email com default: {pessoa.get('email', 'não informado')}")

# inserir/atualizar — O(1)
pessoa["email"] = "joao@email.com"
pessoa["idade"] = 26
print(f"pessoa atualizada: {pessoa}")

# remover — O(1)
del pessoa["cidade"]
print(f"após remover cidade: {pessoa}")

# verificar se chave existe — O(1)
print(f"'nome' existe? {'nome' in pessoa}")
print(f"'cidade' existe? {'cidade' in pessoa}")

nome: João
email: None
email com default: não informado
pessoa atualizada: {'nome': 'João', 'idade': 26, 'cidade': 'São Paulo', 'email': 'joao@email.com'}
após remover cidade: {'nome': 'João', 'idade': 26, 'email': 'joao@email.com'}
'nome' existe? True
'cidade' existe? False


## defaultdict — valor padrão automático

evita ter que verificar se a chave existe antes de acessar.

In [2]:
from collections import defaultdict

# defaultdict com int (default = 0)
contagem: defaultdict[str, int] = defaultdict(int)

palavras = ["maçã", "banana", "maçã", "laranja", "banana", "maçã"]

for palavra in palavras:
    contagem[palavra] += 1  # não precisa verificar se existe

print(f"contagem: {dict(contagem)}")

# defaultdict com list (default = [])
grupos: defaultdict[str, list[str]] = defaultdict(list)

alunos = [("A", "Ana"), ("B", "Bruno"), ("A", "Alice"), ("B", "Bia")]

for turma, nome in alunos:
    grupos[turma].append(nome)  # não precisa inicializar a lista

print(f"grupos: {dict(grupos)}")

contagem: {'maçã': 3, 'banana': 2, 'laranja': 1}
grupos: {'A': ['Ana', 'Alice'], 'B': ['Bruno', 'Bia']}


## Counter — contagem de elementos

especializado para contar ocorrências. já vem com métodos úteis.

In [3]:
from collections import Counter

# contar elementos de uma lista
frutas = ["maçã", "banana", "maçã", "laranja", "banana", "maçã"]
contagem = Counter(frutas)

print(f"contagem: {contagem}")
print(f"mais comuns: {contagem.most_common(2)}")
print(f"total de elementos: {contagem.total()}")

# contar caracteres de uma string
texto = "abracadabra"
chars = Counter(texto)

print(f"\ncontagem de '{texto}': {chars}")

# operações entre Counters
c1 = Counter(["a", "b", "b", "c"])
c2 = Counter(["b", "c", "c", "d"])

print(f"\nc1: {c1}")
print(f"c2: {c2}")
print(f"c1 + c2: {c1 + c2}")
print(f"c1 - c2: {c1 - c2}")  # subtrai, remove zeros e negativos
print(f"c1 & c2 (interseção): {c1 & c2}")  # mínimo de cada
print(f"c1 | c2 (união): {c1 | c2}")  # máximo de cada

contagem: Counter({'maçã': 3, 'banana': 2, 'laranja': 1})
mais comuns: [('maçã', 3), ('banana', 2)]
total de elementos: 6

contagem de 'abracadabra': Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})

c1: Counter({'b': 2, 'a': 1, 'c': 1})
c2: Counter({'c': 2, 'b': 1, 'd': 1})
c1 + c2: Counter({'b': 3, 'c': 3, 'a': 1, 'd': 1})
c1 - c2: Counter({'a': 1, 'b': 1})
c1 & c2 (interseção): Counter({'b': 1, 'c': 1})
c1 | c2 (união): Counter({'b': 2, 'c': 2, 'a': 1, 'd': 1})


## set — conjunto (hashmap só de chaves)

útil para verificar existência e eliminar duplicatas.

In [4]:
# criando um set
numeros = {1, 2, 3, 4, 5}

# adicionar — O(1)
numeros.add(6)

# remover — O(1)
numeros.remove(1)

# verificar existência — O(1)
print(f"3 in numeros: {3 in numeros}")

# eliminar duplicatas de uma lista
lista_com_duplicatas = [1, 2, 2, 3, 3, 3, 4]
sem_duplicatas = list(set(lista_com_duplicatas))
print(f"sem duplicatas: {sem_duplicatas}")

# operações de conjunto
a = {1, 2, 3, 4}
b = {3, 4, 5, 6}

print(f"\na: {a}")
print(f"b: {b}")
print(f"a | b (união): {a | b}")
print(f"a & b (interseção): {a & b}")
print(f"a - b (diferença): {a - b}")
print(f"a ^ b (diferença simétrica): {a ^ b}")

3 in numeros: True
sem duplicatas: [1, 2, 3, 4]

a: {1, 2, 3, 4}
b: {3, 4, 5, 6}
a | b (união): {1, 2, 3, 4, 5, 6}
a & b (interseção): {3, 4}
a - b (diferença): {1, 2}
a ^ b (diferença simétrica): {1, 2, 5, 6}


# padrões comuns com hashmaps

## two sum — encontrar par que soma um valor

problema clássico que usa hashmap para busca O(1).

In [5]:
def two_sum(nums: list[int], target: int) -> tuple[int, int] | None:
    """encontra dois índices cujos valores somam target — O(n)"""
    visto: dict[int, int] = {}  # valor -> índice

    for i, num in enumerate(nums):
        complemento = target - num

        if complemento in visto:
            return (visto[complemento], i)

        visto[num] = i

    return None


nums = [2, 7, 11, 15]
target = 9

resultado = two_sum(nums, target)
print(f"nums: {nums}, target: {target}")
print(f"índices: {resultado}")

if resultado:
    i, j = resultado
    print(f"valores: {nums[i]} + {nums[j]} = {target}")

nums: [2, 7, 11, 15], target: 9
índices: (0, 1)
valores: 2 + 7 = 9


## verificar anagramas

duas palavras são anagramas se têm as mesmas letras com as mesmas frequências.

In [6]:
from collections import Counter


def sao_anagramas(s1: str, s2: str) -> bool:
    """verifica se duas strings são anagramas — O(n)"""
    return Counter(s1.lower()) == Counter(s2.lower())


print(f"'amor' e 'roma': {sao_anagramas('amor', 'roma')}")
print(f"'listen' e 'silent': {sao_anagramas('listen', 'silent')}")
print(f"'hello' e 'world': {sao_anagramas('hello', 'world')}")

'amor' e 'roma': True
'listen' e 'silent': True
'hello' e 'world': False


## agrupar anagramas

In [7]:
from collections import defaultdict


def agrupar_anagramas(palavras: list[str]) -> list[list[str]]:
    """agrupa anagramas juntos — O(n * k log k) onde k = tamanho médio das palavras"""
    grupos: defaultdict[str, list[str]] = defaultdict(list)

    for palavra in palavras:
        # a chave é a palavra ordenada alfabeticamente
        chave = "".join(sorted(palavra))
        grupos[chave].append(palavra)

    return list(grupos.values())


palavras = ["amor", "roma", "mora", "carro", "arco", "roca", "orca"]
grupos = agrupar_anagramas(palavras)

print("grupos de anagramas:")
for grupo in grupos:
    print(f"  {grupo}")

grupos de anagramas:
  ['amor', 'roma', 'mora']
  ['carro']
  ['arco', 'roca', 'orca']


# resumo de complexidade

| operação              | dict/set | notas                              |
| --------------------- | -------- | ---------------------------------- |
| inserir               | O(1)*    | amortizado                         |
| buscar                | O(1)*    | caso médio                         |
| remover               | O(1)*    | caso médio                         |
| iterar                | O(n)     | percorre todos os elementos        |

\* no pior caso (muitas colisões) pode ser O(n), mas é extremamente raro com boas hash functions